# 翻译

人类文明的发展动力之一就是人与人之间的沟通能力。在大多数人类事业中，沟通都非常重要。

![一个使用多种语言的机器人](./images/translation.jpg)

人工智能 (AI) 可以通过翻译不同语言的文本或语音来简化沟通，帮助消除不同国家和地区之间的沟通障碍。

## 创建认知服务资源

在 Azure 中，可以使用认知服务在多种语言间进行翻译。

请按照以下步骤在 Azure 订阅中创建**认知服务**资源（如果还没有该资源）：

> **备注**：如果已有认知服务资源，则只需在 Azure 门户中打开其“**快速入门**”页面，然后将其密钥和终结点复制到下面的单元格中即可。否则，请按照以下步骤创建认知服务资源。

1. 在另一个浏览器标签页中，打开 Azure 门户 (https://portal.azure.com) 并使用 Microsoft 帐户登录。
2. 单击“**&#65291;创建资源**”按钮，搜索“*认知服务*”并以如下设置创建**认知服务**资源：
    * **订阅**： *你的 Azure 订阅*。
    * **资源组**： *选择或创建具有唯一名称的资源组*。
    * **区域**： *选择任何可用区域*：
    * **名称**： *输入一个唯一名称*。
    * **定价层**：S0
    * **我确认我已阅读并理解上述通知**：已选中。
3. 等待部署完成。然后转到认知服务资源，并单击“**概述**”页面上的链接以管理该服务的密钥。你将需要使用终结点和密钥从客户端应用程序连接到认知服务资源。

### 获取认知服务资源的密钥和位置

要使用认知服务资源，需要向客户端应用程序提供其身份验证密钥和位置：

1. 进入 Azure 门户，在认知服务资源的“**密钥和终结点**”页面上复制资源的“**Key1**”，并将其粘贴到以下代码中，替换“**YOUR_COG_KEY**”。
2. 复制资源的**位置**并将其粘贴到以下代码中，替换“**YOUR_COG_LOCATION**”。
>**备注**：停留在“**密钥和终结点**”页面上，并从其中复制“**位置**”（例如：_westus_）。请*不要*在“位置”字段的字词之间添加空格。 
3. 通过单击位于单元格左侧的“**运行单元格**”(&#9655;) 按钮运行下方代码。

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_location = 'YOUR_COG_LOCATION'

print('Ready to use cognitive services in {} using key {}'.format(cog_location, cog_key))

## 翻译文本

顾名思义，**文本翻译**服务可用于将一种语言的文本翻译为另一种语言。

没有适用于此服务的 Python SDK，但可以使用其 REST 接口通过 HTTP 向终结点提交**请求**，使用 Python 通过请求库完成此操作相对简单。有关待翻译文本和所生成译文的信息采用 JSON 格式进行交换。

运行以下单元格以创建一个执行此操作的函数，然后通过一次简单的英语翻译为法语来测试该函数。

In [ ]:
# Create a function that makes a REST request to the Text Translation service
def translate_text(cog_location, cog_key, text, to_lang='fr', from_lang='en'):
    import requests, uuid, json

    # Create the URL for the Text Translator service REST request
    path = 'https://api.cognitive.microsofttranslator.com/translate?api-version=3.0'
    params = '&from={}&to={}'.format(from_lang, to_lang)
    constructed_url = path + params

    # Prepare the request headers with Cognitive Services resource key and region
    headers = {
        'Ocp-Apim-Subscription-Key': cog_key,
        'Ocp-Apim-Subscription-Region':cog_location,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }

    # Add the text to be translated to the body
    body = [{
        'text': text
    }]

    # Get the translation
    request = requests.post(constructed_url, headers=headers, json=body)
    response = request.json()
    return response[0]["translations"][0]["text"]


# Test the function
text_to_translate = "Hello"

translation = translate_text(cog_location, cog_key, text_to_translate, to_lang='fr', from_lang='en')
print('{} -> {}'.format(text_to_translate,translation))

该服务应该将英语文本“Hello”翻译为法语文本“Bonjour”。

请注意，指定语言时使用的是标准语言缩写，*en* 代表英语，*fr* 代表法语。也可以使用包含特定区域的缩写，这适用于在不同地理区域使用同一种语言的情况（通常采用不同的拼写形式）。例如，*en-US* 表示美式英语，而 *en-GB* 表示英式英语。

运行下方单元格以在英式英语和意大利语之间进行翻译。

In [ ]:
text_to_translate = "Hello"

translation = translate_text(cog_location, cog_key, text_to_translate, to_lang='it-IT', from_lang='en-GB')
print('{} -> {}'.format(text_to_translate,translation))

让我们尝试另一种翻译，将美式英语翻译为中文。

In [ ]:
text_to_translate = "Hello"

translation = translate_text(cog_location, cog_key, text_to_translate, to_lang='zh-CN', from_lang='en-US')
print('{} -> {}'.format(text_to_translate,translation))

## 语音翻译

可以使用**语音**服务来翻译口语。

现在可以运行以下单元格来创建并测试一个函数，该函数使用语音 SDK 来翻译语音。

> **备注**：你还需要使用扬声器来收听音频。

In [ ]:
from playsound import playsound 

# Create a function to translate audio in one language to text in another
def translate_speech(cog_location, cog_key, audio_file=None, to_lang='fr-FR', from_lang='en-US'):
    from azure.cognitiveservices.speech import SpeechConfig, AudioConfig, ResultReason
    from azure.cognitiveservices.speech.translation import SpeechTranslationConfig, TranslationRecognizer

    # Configure the speech translation service
    translation_config = SpeechTranslationConfig(subscription=cog_key, region=cog_location)
    translation_config.speech_recognition_language = from_lang
    translation_config.add_target_language(to_lang)

    # Configure audio input
    if audio_file is None:
        audio_config = AudioConfig() # Use default input (microphone)
    else:
        audio_config = AudioConfig(filename=audio_file) # Use file input

    # Create a translation recognizer and use it to translate speech input
    recognizer = TranslationRecognizer(translation_config, audio_config)
    result = recognizer.recognize_once()

    # Did we get it?
    translation = ''
    speech_text = ''
    if result.reason == ResultReason.TranslatedSpeech:
        speech_text = result.text
        translation =  result.translations[to_lang]
    elif result.reason == ResultReason.RecognizedSpeech:
        speech_text = result.text
        translation =  'Unable to translate speech'
    else:
        translation = 'Unknown'
        speech_text = 'Unknown'

    # rturn the translation
    return speech_text, translation
    

# Test the function
import os

file_name = 'english.wav'
file_path = os.path.join('data', 'translation', file_name)
speech, translated_speech = translate_speech(cog_location, cog_key, file_path, to_lang='es', from_lang='en-US')
result = '{} -> {}'.format(speech, translated_speech)

# Show translated text
playsound(file_path)
print(result)

请注意，必须使用 2 个字符的语言代码（例如“*en*”）来确定目标语言，而源语言必须包括区域指示符（例如“*en-US*”）。

我们来尝试将法语翻译为英语。

In [ ]:
from playsound import playsound
import os

file_name = 'french.wav'
file_path = os.path.join('data', 'translation', file_name)
speech, translated_speech = translate_speech(cog_location, cog_key, file_path, to_lang='en', from_lang='fr-FR')
result = '{} -> {}'.format(speech, translated_speech)

# Show translated text
playsound(file_path)
print(result)

## 了解详细信息

可以在服务文档中详细了解[文本翻译](https://docs.microsoft.com/azure/cognitive-services/translator/)以及[使用语音服务进行翻译](https://docs.microsoft.com/azure/cognitive-services/speech-service/index-speech-translation)。